## <i><u> Proceed to Part 2 for looking merging Data and coordinats on PostCodes </u></i>

# Part 1. Grabing the Table from html to DataFrame object, parsing and wrangling

## <i><u> Proceed to Part 2 for looking merging Data and coordinats on PostCodes </u></i>

## 1. Installation and Import of libraries

In [15]:
import random # library for random number generation
import requests # library to handle requests
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


In [2]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.6.3       |           py35_0         139 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0        conda-forge
    ca-certificates: 2019.1.23-0          --> 2019.3.9-hecc5488_0 conda-forge
    certifi:         2018.8.24-py35_1     --> 2018.8.24-py35_1001 

In [16]:
from bs4 import BeautifulSoup #library for work with html
print('Soup is ready, ser!')

Soup is ready, ser!


In [4]:
!conda install -c conda-forge lxml --yes #library for parsing html

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxml2-2.9.9              |       h13577e0_0         2.0 MB  conda-forge
    lxml-4.2.5                 |   py35hc9114bc_0         5.8 MB  conda-forge
    libxslt-1.1.32             |    h4785a14_1002         539 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.3 MB

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.9-h13577e0_0     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h4785a14_1002 conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge


libxml2-2.9.9        | 2.0 MB    | ##################################### | 100% 
lxml-4.2.5    

## 2. Grabbing the Table and parsing it to the DataFrame object "data"

In [17]:
#Get html to "soup"
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#source = requests.get('https://coreyms.com').text
soup = BeautifulSoup(source, 'lxml')

In [18]:
#Get table of postcodes to "table"
table = soup.find('table')

In [19]:
#Get headers by teg 'th'
headers = [col.text for col in table.find('tr').find_all('th')]

#Get rows by teg 'tr'
rows = table.find_all('tr')

#Get values by teg 'td' inside rows
values = [[cell.text for cell in row.find_all('td')] for row in rows]
data = pd.DataFrame(columns = headers, data = values)
data.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


## 3. Data wrangling

In [20]:
# Drop the first "None" line
data.drop(index = 0, inplace = True)
data.reset_index(inplace = True, drop = True)
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [21]:
#Replace \n
data.columns = data.columns.str.replace('\n','')
data['Neighbourhood'] = data['Neighbourhood'].str.replace('\n','')
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [22]:
#Drop rows with 'Not assigned' Borough
data = data.loc[data['Borough'] != 'Not assigned']
data.reset_index(inplace = True, drop = True)
data.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [23]:
#Rename 'Not assigned' Neighbourhood with name of Borough
data.loc[data['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = data.loc[data['Neighbourhood'] == 'Not assigned', 'Borough']
data.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [24]:
data.sort_values(by=['Postcode'], inplace=True)
data.reset_index(inplace = True, drop = True)
data.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [25]:
#Concatinating rows with the same Postcode
ind = 1
while ind < len(data):
    if (data.loc[ind, 'Postcode']==data.loc[ind-1, 'Postcode']):
        data.loc[ind-1, 'Neighbourhood'] = data.loc[ind-1, 'Neighbourhood']+', '+ data.loc[ind, 'Neighbourhood']
        data.drop(ind, inplace = True)
        data.reset_index(inplace = True, drop = True)
    else:
        ind += 1
data.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [26]:
data.shape

(103, 3)

# Part II. Adding the latitude and the longtitude to the Data

In [27]:
#load csv file with coodrinates to a DataFrame object "data_ll"
data_ll = pd.read_csv('http://cocl.us/Geospatial_data')
data_ll.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [31]:
#Preparing data for merging
data_ll.rename(columns={'Postal Code':'Postcode'}, inplace = True)
print(data_ll.columns)
print(data.columns)

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')
Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')


In [30]:
#Mergind datas on Postcode
data = data.merge(data_ll, on='Postcode')
data.head(10) 

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848
